<div style="display: flex; background-color: RGB(255,114,0);" >
<h1 style="margin: auto; padding: 30px; ">Brief 9 -  </h1>
</div>

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); "> Importation des librairies</h3>
</div>

In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score 
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score, make_scorer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection  import cross_val_score
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge  
from sklearn.linear_model import Lasso  
from xgboost import XGBRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from math import sqrt
warnings.filterwarnings("ignore")

In [4]:
data = pd.read_csv(r'../../data/CURATED/df_final.csv', encoding="UTF8", sep=',')

In [5]:
df = data.copy()
df.drop(columns=['Id'], inplace=True)

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); "> Train and Test Splits</h3>
</div>

In [6]:
for col in ['valeur','surf_bati','surf terr']:
    df[col]=np.log10(df[col])
df.replace([np.inf, -np.inf], -1, inplace=True) 


In [7]:
scalery = MaxAbsScaler()
scalerx = MaxAbsScaler()

zz = pd.get_dummies(df['local'])
yy = pd.DataFrame(df['valeur'])
XX = df.drop(['local','valeur'], axis=1)

y = scalery.fit_transform(yy.values)
X = pd.concat([pd.DataFrame(scalerx.fit_transform(XX), columns= XX.columns),zz], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); "> Model-1 Linear Regression </h3>
</div>

In [6]:
regresseur = LinearRegression(fit_intercept=True)
regresseur.fit(X_train,y_train)
y_pred=regresseur.predict(X_test)

In [7]:
print(" Results for linear Regression:")
print("*******************************")
print('MAE:', mae(y_test, y_pred))
print('R-squared:', r2_score(y_test, y_pred))
print('RMSE:', np.sqrt(mse(y_test, y_pred)))

 Results for linear Regression:
*******************************
MAE: 0.016388553809028254
R-squared: 0.627540245059448
RMSE: 0.023612547405599617


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); "> Model-2 Ridge Regression </h3>
</div>

In [8]:
params = {'alpha': [0.01, 0.05, 0.1, 0.5,1,10]}
model = GridSearchCV(Ridge(), params, scoring = 'neg_mean_absolute_error', cv=10, n_jobs=-1)
model.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=Ridge(), n_jobs=-1,
             param_grid={'alpha': [0.01, 0.05, 0.1, 0.5, 1, 10]},
             scoring='neg_mean_absolute_error')

In [17]:
model.best_estimator_

Ridge(alpha=0.01)

In [14]:
y_train_pred = model.predict(X_train)
y_pred = model.predict(X_test)

In [15]:
#Train
print("Train Results for Ridge Regression:")
print("*******************************")
print("RMSE: ", sqrt(mse(y_train, y_train_pred)))
print("R-squared: ", r2_score(y_train, y_train_pred))
print("MAE: ", mae(y_train, y_train_pred))

Train Results for Ridge Regression:
*******************************
RMSE:  0.023966772856505277
R-squared:  0.6205538277005892
MAE:  0.016487763390486864


In [16]:
#Test
print("Test Results for Ridge Regression:")
print("*******************************")
print("RMSE: ", sqrt(mse(y_test, y_pred)))
print("R-squared: ", r2_score(y_test, y_pred))
print("MAE: ", mae(y_test, y_pred))

Test Results for Ridge Regression:
*******************************
RMSE:  0.023845463136974266
R-squared:  0.6256679665534641
MAE:  0.016467661855520094


In [19]:
model = Ridge(alpha=0.01)
model.fit(X_train, y_train)

Ridge(alpha=0.01)

In [20]:
indices = np.argsort(-abs(model.coef_))
print("The features in order of importance are:")
print(50*'-')
for feature in X.columns[indices]:
    print(feature)

The features in order of importance are:
--------------------------------------------------
['nbr_piece' 'Appartement' 'tot_lot' 'Dépendance'
 'Local industriel. commercial ou assimilé' 'Maison' 'Terrain' 'prix_m2'
 'surf_bati' 'Terrain' 'Seul' 'Lotissement' 'surf terr' 'Batiment'
 'typ_lot' 'Centre commercial' 'mix']


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); "> Model-3 Lasso Regression </h3>
</div>

In [15]:
params = {'alpha': [1000, 10000]}
model = GridSearchCV(Lasso(), params, scoring = 'neg_mean_absolute_error', cv=20, n_jobs=-1)
model.fit(X_train, y_train)

GridSearchCV(cv=20, estimator=Lasso(), n_jobs=-1,
             param_grid={'alpha': [1000, 10000]},
             scoring='neg_mean_absolute_error')

In [16]:
model.best_estimator_

Lasso(alpha=1000)

In [17]:
y_train_pred = model.predict(X_train)
y_pred = model.predict(X_test)

In [18]:
#Train
print("Train Results for Lasso Regression:")
print("*******************************")
print("RMSE: ", sqrt(mse(y_train, y_train_pred)))
print("R-squared: ", r2_score(y_train, y_train_pred))
print("MAE: ", mae(y_train, y_train_pred))

Train Results for Lasso Regression:
*******************************
RMSE:  0.03897820677856909
R-squared:  0.0
MAE:  0.029475223142339406


In [19]:
#Test
print("Test Results for Lasso Regression:")
print("*******************************")
print("RMSE: ", sqrt(mse(y_test, y_pred)))
print("R-squared: ", r2_score(y_test, y_pred))
print("MAE: ", mae(y_test, y_pred))

Test Results for Lasso Regression:
*******************************
RMSE:  0.038691136125667554
R-squared:  -3.8298600816277784e-05
MAE:  0.029268858987282747


In [20]:
model = Lasso(alpha=1000)
model.fit(X_train, y_train)

Lasso(alpha=1000)

In [21]:
indices = np.argsort(-abs(model.coef_))
print("The features in order of importance are:")
print(50*'-')
for feature in X.columns[indices]:
    print(feature)

The features in order of importance are:
--------------------------------------------------
Appartement
Seul
Lotissement
Centre commercial
Batiment
prix_m2
surf terr
Terrain
surf_bati
tot_lot
nbr_piece
Terrain
Maison
Local industriel. commercial ou assimilé
Dépendance
typ_lot
mix


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); "> Model-4 Random Forest Regression </h3>
</div>

In [22]:
acc_scorer = make_scorer(accuracy_score)
params = {'n_estimators': [50,100], 'min_samples_split': [5, 10], 'min_samples_leaf': [2, 4], 'max_depth': [10,15]}
model =  GridSearchCV(RandomForestRegressor(), params, 
                           scoring = acc_scorer, cv=5, n_jobs=-1)

In [ ]:
model.best_estimator_

In [29]:
model.fit(X_train, y_train)

RandomForestRegressor(min_samples_leaf=2, min_samples_split=5,
                      n_estimators=1000)

In [28]:
model = RandomForestRegressor(min_samples_leaf=2, min_samples_split=5, n_estimators=1000)

In [30]:
y_train_pred = model.predict(X_train)
y_pred = model.predict(X_test)

In [31]:
#Train
print("Train Results for RandomForest Regression:")
print("*******************************")
print("RMSE: ", sqrt(mse(y_train, y_train_pred)))
print("R-squared: ", r2_score(y_train, y_train_pred))
print("MAE: ", mae(y_train, y_train_pred))

Train Results for RandomForest Regression:
*******************************
RMSE:  0.0003009330654278548
R-squared:  0.9999403931587674
MAE:  3.2565236412892644e-05


In [32]:
#Test
print("Test Results for RandomForest Regression:")
print("*******************************")
print("RMSE: ", sqrt(mse(y_test, y_pred)))
print("R-squared: ", r2_score(y_test, y_pred))
print("MAE: ", mae(y_test, y_pred))

Test Results for RandomForest Regression:
*******************************
RMSE:  0.0004723171899729796
R-squared:  0.9998509742996404
MAE:  5.692348164098807e-05


In [ ]:
indices = np.argsort(-model.feature_importances_)
print("The features in order of importance are:")
print(50*'-')
for feature in X.columns[indices]:
    print(feature)

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); "> Model-5 Descision Tree Regression </h3>
</div>

In [ ]:
params = {'min_samples_split': [2,4], 'min_samples_leaf': [5,10], 'max_depth': [10,15]}
model = RandomizedSearchCV(DecisionTreeRegressor(), params, n_iter=20, scoring = 'neg_mean_absolute_error', cv=10, n_jobs=-1)
model.fit(X_train, y_train)

In [ ]:
model.best_estimator_

In [ ]:
y_train_pred = model.predict(X_train)
y_pred = model.predict(X_test)

In [ ]:
#Train
print("Train Results for DecisionTree Regression:")
print("*******************************")
print("RMSE: ", sqrt(mse(y_train, y_train_pred)))
print("R-squared: ", r2_score(y_train, y_train_pred))
print("MAE: ", mae(y_train, y_train_pred))

In [ ]:
#Test
print("Test Results for DecisionTree Regression:")
print("*******************************")
print("RMSE: ", sqrt(mse(y_test, y_pred)))
print("R-squared: ", r2_score(y_test, y_pred))
print("MAE: ", mae(y_test, y_pred))

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); "> Model-6 KNN Regression </h3>
</div>

In [ ]:
# creating odd list of K for KNN
neighbors = list(range(1,50,2))
# empty list that will hold cv scores
cv_scores = []

# perform 10-fold cross validation
for k in neighbors:
    knn = KNeighborsRegressor(n_neighbors=k)
    scores = cross_val_score(knn, X_train, y_train, cv=10, scoring='neg_mean_absolute_error')
    cv_scores.append(scores.mean())

# changing to misclassification error
MSE = [1 - x for x in cv_scores]

# determining best k
optimal_k = neighbors[MSE.index(min(MSE))]
print('\nThe optimal number of neighbors is %d.' % optimal_k)

In [ ]:
model = KNeighborsRegressor(n_neighbors = optimal_k)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_pred = model.predict(X_test)

In [ ]:
#Train
print("Train Results for KNN Regression:")
print("*******************************")
print("RMSE: ", sqrt(mse(y_train.values, y_train_pred)))
print("R-squared: ", r2_score(y_train.values, y_train_pred))
print("MAE: ", mae(y_train.values, y_train_pred))

In [ ]:
#Test
print("Test Results for KNN Regression:")
print("*******************************")
print("RMSE: ", sqrt(mse(y_test, y_pred)))
print("R-squared: ", r2_score(y_test, y_pred))
print("MAE: ", mae(y_test, y_pred))

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); "> Model-7 XGBoost Regression </h3>
</div>

In [49]:
params = {'max_depth': [10,15], 'learning_rate': [0.01, 0.1], 'n_estimators': [500,600], 'reg_lambda': [100.0]}
model = RandomizedSearchCV(XGBRegressor(), params, n_iter=20, scoring = 'neg_mean_absolute_error', cv=5, n_jobs=-1)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_train_pred = model.predict(X_train)
y_pred = model.predict(X_test)

yert = model.predict(X_test[0].reshape(1, -1))

In [ ]:
#Train
print("Train Results for XGBoost Regression:")
print("*******************************")
print("RMSE: ", sqrt(mse(y_train, y_train_pred)))
print("R-squared: ", r2_score(y_train, y_train_pred))
print("MAE: ", mae(y_train, y_train_pred))

In [ ]:
#Test
print("Test Results for XGBoost Regression:")
print("*******************************")
print("RMSE: ", sqrt(mse(y_test, y_pXGBoostred)))
print("R-squared: ", r2_score(y_test, y_pred))
print("MAE: ", mae(y_test, y_pred))

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); "> Model-7 SGD Regression </h3>
</div>

In [ ]:
SGD = SGDRegressor(max_iter = 100)
SGD.fit(X, y)
r2_score(SGD.predict(X), y)